In [1]:
import ROOT
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x444b7e0


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [5]:
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools_old.h\"")

True

In [6]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

# ROOT.OOTools.init(250, mW, mZ)
ROOT.OOTools.init(190, mW, mZ)

In [7]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)
ROOT.ildStyle.SetPalette(ROOT.kBird)

In [8]:
%%cpp
using namespace ROOT::VecOps;

In [9]:
n_points = 5000000
# n_points = 1000000

df = ROOT.RDataFrame(n_points)

In [10]:
df = df.Define("co", "gRandom->Uniform(-1., 1.)")
df = df.Define("co1", "gRandom->Uniform(-1., 1.)")
df = df.Define("co2", "gRandom->Uniform(-1., 1.)")
df = df.Define("ph1", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")
df = df.Define("ph2", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")

In [11]:
g1 = 0.0002
g2 = 0.0004

df = df.Define("S_0", "OOTools::proba(co, co1, co2, ph1, ph2, 0, true, 0)")
df = df.Define("S_1_1", "OOTools::proba(co, co1, co2, ph1, ph2, 1, false, 1)")
df = df.Define("S_1_2", "OOTools::proba(co, co1, co2, ph1, ph2, 2, false, 1)")
df = df.Define("w_proba", "S_0")

df = df.Define("S", f"S_0 + {g1} * S_1_1 + {g2} * S_1_2")
# FIXME: this statement is not completely true, according to Markus comment in the fortran code
# C       d(sigma) =  (alpha/z)**2 *(be/s)* (3/64)**2 *(1/ (2*Pi))* proba*
# C            (appropriate form factor if not zeroth order)*
# C            d(phase space)
# C     with
# C       d(phase space) = d(co) * d(co1)*d(phi1) * d(co2)*d(phi2) ,
# C     the fine structure constant alpha and z,be,s defined as in
# C     suboutine init.
df = df.Define("d_sigma", "S")
df = df.Vary("w_proba", "RVec({d_sigma})", ["fz1"], "aTGC")
df = df.Define("O_1", "S_1_1 / S_0")
df = df.Define("O_2", "S_1_2 / S_0")

# as we are sampling phase space uniformly we can also get rid of weighting by S and just use
# these quantities to calculate g_i
# g_i = c_ij^-1 * (a_j - b_j)
# with c_ij = S_1_i * S_1_j / S_0
# a_j = S_1_j * S / S_0
# b_j = S_1_j
# for j in range(1, 3):
#     df = df.Define(f"a_{j}", f"S_1_{j} * S / S_0")
#     df = df.Define(f"b_{j}", f"S_1_{j}")
#     for i in range(1, 3):
#         df = df.Define(f"c_{i}{j}", f"S_1_{i} * S_1_{j} / S_0")
# now try to calculate the same but actually using the optimal observables themselves and weighting by S or S_0 when appropriate
# a needs to be weighted by S, b and c by S_0
for j in range(1, 3):
    df = df.Define(f"a_{j}", f"O_{j} * S")
    df = df.Define(f"b_{j}", f"O_{j} * S_0")
    for i in range(1, 3):
        df = df.Define(f"c_{i}{j}", f"O_{i} * O_{j} * S_0")

In [12]:
m_a = []
m_b = []
m_c = []

for j in range(1, 3):
    # m_a.append(df.Mean(f"a_{j}"))
    # m_b.append(df.Mean(f"b_{j}"))
    m_a.append(df.Sum(f"a_{j}"))
    m_b.append(df.Sum(f"b_{j}"))
    for i in range(1, 3):
        # m_c.append(df.Mean(f"c_{i}{j}"))
        m_c.append(df.Sum(f"c_{i}{j}"))

In [13]:
pi = ROOT.Math.Pi()

nbins = 40

h_co  = df.Histo1D(("", ";co", nbins, -1., 1.), "co", "w_proba")
h_co1 = df.Histo1D(("", ";co1", nbins, -1., 1.), "co1", "w_proba")
h_co2 = df.Histo1D(("", ";co2", nbins, -1., 1.), "co2", "w_proba")
h_ph1 = df.Histo1D(("", ";ph1", nbins, -pi, pi), "ph1", "w_proba")
h_ph2 = df.Histo1D(("", ";ph2", nbins, -pi, pi), "ph2", "w_proba")

v_co = ROOT.RDF.Experimental.VariationsFor(h_co)
v_co1 = ROOT.RDF.Experimental.VariationsFor(h_co1)
v_co2 = ROOT.RDF.Experimental.VariationsFor(h_co2)
v_ph1 = ROOT.RDF.Experimental.VariationsFor(h_ph1)
v_ph2 = ROOT.RDF.Experimental.VariationsFor(h_ph2)

h_weight = df.Histo1D("w_proba")
h_O_1 = df.Histo1D(("", ";O = S_1_1 / S_0", 100, -1.5, 1.5), "O_1", "w_proba")
v_O_1 = ROOT.RDF.Experimental.VariationsFor(h_O_1)
h_O_2 = df.Histo1D(("", ";O = S_1_2 / S_0", 100, -1.5, 1.5), "O_2", "w_proba")
v_O_2 = ROOT.RDF.Experimental.VariationsFor(h_O_2)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xb145090


In [14]:
h_2D = {}
from itertools import combinations
angles = ["co", "co1", "co2", "ph1", "ph2"]
for a, b in combinations(angles, 2):
    k = f"{a}_{b}"
    bin_end_a = pi if a.startswith("ph") else 1.
    bin_end_b = pi if b.startswith("ph") else 1.
    h_model = ("", f";{a};{b}", nbins, -bin_end_a, bin_end_a, nbins, -bin_end_b, bin_end_b)
    h_2D[k] = df.Histo2D(h_model, a, b, "w_proba")

In [15]:
a = np.asarray([m.GetValue() for m in m_a])
b = np.asarray([m.GetValue() for m in m_b])
c = np.asarray([m.GetValue() for m in m_c]).reshape((2, 2))
print(a)
print(b)
print(c)
g = np.linalg.solve(c, a-b)
print(g)

[ 1745880.00336001 -1405373.72525577]
[ 1746640.45633036 -1407228.23521632]
[[ 7982859.13843773 -5892561.99510085]
 [-5892561.99510085  7582555.89890372]]
[0.0002 0.0004]


In [16]:
c_O_1 = ROOT.TCanvas()
h_O_1.Draw()
v_O_1["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_O_1["aTGC:fz1"].Draw("same)")
c_O_1.Draw()

c_O_2 = ROOT.TCanvas()
h_O_2.Draw()
v_O_2["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_O_2["aTGC:fz1"].Draw("same)")
c_O_2.Draw()

c_co = ROOT.TCanvas()
h_co.SetMinimum(0)
h_co.Draw()
v_co["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_co["aTGC:fz1"].Draw("same)")
c_co.Draw()

c_co1 = ROOT.TCanvas()
h_co1.SetMinimum(0)
h_co1.Draw()
v_co1["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_co1["aTGC:fz1"].Draw("same)")
c_co1.Draw()

c_ph1 = ROOT.TCanvas()
h_ph1.SetMinimum(0)
h_ph1.Draw()
v_ph1["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_ph1["aTGC:fz1"].Draw("same)")
c_ph1.Draw()

c_co2 = ROOT.TCanvas()
h_co2.SetMinimum(0)
h_co2.Draw()
v_co2["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_co2["aTGC:fz1"].Draw("same)")
c_co2.Draw()

c_ph2 = ROOT.TCanvas()
h_ph2.SetMinimum(0)
h_ph2.Draw()
v_ph2["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_ph2["aTGC:fz1"].Draw("same)")
c_ph2.Draw()

c_weight = ROOT.TCanvas()
h_weight.SetMinimum(0)
h_weight.Draw()
c_weight.Draw()

In [17]:
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPalette(ROOT.kBird)
ROOT.gStyle.SetNumberContours(20)
c_2D = {}
for a, b in combinations(angles, 2):
    k = f"{a}_{b}"
    c_2D[k] = ROOT.TCanvas()
    h_2D[k].Draw("colz0")
    c_2D[k].Draw()